## Evaluation function to calculate the value of each posible solution distribution

For all algorithms, we need to calculate the cost total cost of active edges, and reliability of each node to be greater than 1.

The solution is a dictionary with total_cost and dict of reliabilities with the following structure:

```python
{
    'total_cost': number of the total cost of the edges selected,
    'relabilities': {
        0: (1.1, 2)
        1: (0.4, 1)
        ...
    }
}
```

In relibilities dict, the key is the node id, and the value is a tuple with the reliability and the number of edges that are connected to the node.

In [2]:
def is_reliable(solution: dict) -> bool:
    """
    Evaluate if actual solution is reliable for each node.
    Parameters:
    ----------
    solution: dict
        Dictionary with the solution of the problem, where the key is the node and the value is the selected edge.

    Returns:
    -------
    bool
        True if the solution is reliable, False otherwise.

    """
    # Check if each node on solution have a realiable value greater than 1
    for rel in solution['relabilities'].values():
        if rel < 1:
            return False

    return True

In [3]:
# testing the function
true_solution = {
    'total_cost': 10,
    'relabilities': {
        0: 1.0,
        1: 1.9,
        2: 1.3,
        3: 1.0,
        4: 1.2
    }
}

false_solution = {
    'total_cost': 10,
    'relabilities': {
        0: 1.0,
        1: 0.9,
        2: 1.0,
        3: 1.0,
        4: 0.8
    }
}

print(is_reliable(true_solution))
print(is_reliable(false_solution))

True
False


## graph generator

In [4]:
import networkx as nx
import numpy as np

# Specific problem graph generator
def generate_random_graph(num_nodes: int, min_cost: int, max_cost: int):
    """
    Undirected graph generator, where the graph is fully connected
    and the weight of the edges are random cost and a reliability value.
    Parameters:
    ----------
    num_nodes: int
        Number of nodes in the graph
    min_cost: int
        Minimum value of the cost of the edges
    max_cost: int
        Maximum value of the cost of the edges
    """
    # num_edges to have a full connected graph
    num_edges = num_nodes * (num_nodes - 1) // 2
    # generate a random graph with num_nodes nodes and num_edges edges
    graph = nx.gnm_random_graph(num_nodes, num_edges)
    for u, v in graph.edges:
        # cost and reliability of each edge connection beetween nodes
        graph[u][v]['values'] = {
            "cost": np.random.randint(min_cost, max_cost),
            "reliability": np.random.uniform(0.3, 1),
        }
    return graph

In [5]:
# testing the function
base_graph = generate_random_graph(num_nodes=5, min_cost=1, max_cost=20)

# print matrix with value tuples
print("Graph Nodes:")
print(base_graph.nodes)


print("\nGraph Edges with Attributes:")
for u, v, attr in base_graph.edges(data=True):
    # u and v are integers representing the nodes
    print(f"({u}, {v}) -> cost: {attr['values']['cost']}, reliability: {attr['values']['reliability']:.2f}")

# print the nodes in order.
print("\nGraph Nodes in order:")
for node in base_graph.nodes:
    print(node)

Graph Nodes:
[0, 1, 2, 3, 4]

Graph Edges with Attributes:
(0, 1) -> cost: 9, reliability: 0.33
(0, 2) -> cost: 9, reliability: 0.48
(0, 3) -> cost: 17, reliability: 0.46
(0, 4) -> cost: 7, reliability: 0.47
(1, 2) -> cost: 6, reliability: 0.48
(1, 3) -> cost: 3, reliability: 0.82
(1, 4) -> cost: 14, reliability: 0.96
(2, 3) -> cost: 15, reliability: 0.54
(2, 4) -> cost: 16, reliability: 0.76
(3, 4) -> cost: 13, reliability: 0.56

Graph Nodes in order:
0
1
2
3
4


## Problem description

Una organizacion necesita disenar una red de comunicacion robusta contra fallos como respaldo para recuperacion de desastres. La construccion de esta red se modela de la siguiente forma:

Cada potencial enlace entre los nodos iniciales tiene:

* Un costo mayor a cero
* Una calificacion de fiabilidad del cero al uno
 
A partir de estas caracteristicas, para cualquier numero de nodos iniciales con potenciales enlaces, se debe construir una red donde:

* Debe existir algun camino entre cualesquiera dos nodos de la red
* Se minimice el costo total de todos los enlaces en la red
* La suma de la fiabilidad de todos los enlaces a cada nodo sea mayor o igual a uno

## Algorithms


### Brute force 



In [6]:
edges = [(0, 1), (1, 2), (2, 3)]
for r in range(1, len(edges) + 1):
    print(f"r = {r}")


r = 1
r = 2
r = 3


In [7]:
from itertools import combinations
edges = [(0, 1), (1, 2), (2, 3)]
for subset in combinations(edges, 2):  # r = 2
    print(subset)


((0, 1), (1, 2))
((0, 1), (2, 3))
((1, 2), (2, 3))


#### Testing for iterate over all possible edges combinations

In [8]:

from itertools import combinations
import networkx as nx

nodes = [0, 1, 2, 3]
# build a fully connected graph
edges = [
    (0, 1, {"cost": 5, "reliability": 0.9}),
    (0, 2, {"cost": 4, "reliability": 0.95}),
    (0, 3, {"cost": 3, "reliability": 0.8}),
    (1, 2, {"cost": 7, "reliability": 0.8}),
    (1, 3, {"cost": 2, "reliability": 0.9}),
    (2, 3, {"cost": 6, "reliability": 0.85}),
]

In [9]:

for r in range(1, len(edges) + 1):  # De 1 a todas las aristas
    for subset in combinations(edges, r):
        subgraph = nx.Graph()
        subgraph.add_nodes_from(nodes)
        subgraph.add_edges_from([(u, v, data) for u, v, data in subset])
        print(f"Subgrafo con {r} aristas:", subgraph.edges())


Subgrafo con 1 aristas: [(0, 1)]
Subgrafo con 1 aristas: [(0, 2)]
Subgrafo con 1 aristas: [(0, 3)]
Subgrafo con 1 aristas: [(1, 2)]
Subgrafo con 1 aristas: [(1, 3)]
Subgrafo con 1 aristas: [(2, 3)]
Subgrafo con 2 aristas: [(0, 1), (0, 2)]
Subgrafo con 2 aristas: [(0, 1), (0, 3)]
Subgrafo con 2 aristas: [(0, 1), (1, 2)]
Subgrafo con 2 aristas: [(0, 1), (1, 3)]
Subgrafo con 2 aristas: [(0, 1), (2, 3)]
Subgrafo con 2 aristas: [(0, 2), (0, 3)]
Subgrafo con 2 aristas: [(0, 2), (1, 2)]
Subgrafo con 2 aristas: [(0, 2), (1, 3)]
Subgrafo con 2 aristas: [(0, 2), (2, 3)]
Subgrafo con 2 aristas: [(0, 3), (1, 2)]
Subgrafo con 2 aristas: [(0, 3), (1, 3)]
Subgrafo con 2 aristas: [(0, 3), (2, 3)]
Subgrafo con 2 aristas: [(1, 2), (1, 3)]
Subgrafo con 2 aristas: [(1, 2), (2, 3)]
Subgrafo con 2 aristas: [(1, 3), (2, 3)]
Subgrafo con 3 aristas: [(0, 1), (0, 2), (0, 3)]
Subgrafo con 3 aristas: [(0, 1), (0, 2), (1, 2)]
Subgrafo con 3 aristas: [(0, 1), (0, 2), (1, 3)]
Subgrafo con 3 aristas: [(0, 1), (0, 2),

#### Testing for iterate over all possible edges combinations with each node having at least two edges

In [10]:
# bool function to check if every node has two edges
def are_all_nodes_at_least_two_edges(graph: nx.Graph) -> bool:
    """
    Check if every node has two edges
    """
    # Check if every node has two edges
    for node in graph.nodes:
        if graph.degree(node) < 2:
            return False
    return True

In [11]:

for r in range(1, len(edges) + 1):  # De 1 a todas las aristas
    for subset in combinations(edges, r):
        subgraph = nx.Graph()
        subgraph.add_nodes_from(nodes)
        subgraph.add_edges_from([(u, v, data) for u, v, data in subset])
        if are_all_nodes_at_least_two_edges(subgraph):
            print(f"Subgrafo con {r} aristas:", subgraph.edges())


Subgrafo con 4 aristas: [(0, 1), (0, 2), (1, 3), (2, 3)]
Subgrafo con 4 aristas: [(0, 1), (0, 3), (1, 2), (2, 3)]
Subgrafo con 4 aristas: [(0, 2), (0, 3), (1, 2), (1, 3)]
Subgrafo con 5 aristas: [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)]
Subgrafo con 5 aristas: [(0, 1), (0, 2), (0, 3), (1, 2), (2, 3)]
Subgrafo con 5 aristas: [(0, 1), (0, 2), (0, 3), (1, 3), (2, 3)]
Subgrafo con 5 aristas: [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3)]
Subgrafo con 5 aristas: [(0, 1), (0, 3), (1, 2), (1, 3), (2, 3)]
Subgrafo con 5 aristas: [(0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
Subgrafo con 6 aristas: [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


### Brute force algorithm

In [12]:
import networkx as nx
import numpy as np
from itertools import combinations

def brute_force_reliable_network(graph: nx.Graph) -> dict:
    """
    Solve the reliable network problem using brute force.

    Parameters:
    ----------
    graph: nx.Graph
        Graph with nodes and edges containing 'cost' and 'reliability'.

    Returns:
    -------
    dict
        The best solution with minimum cost and sufficient reliability for all nodes.
    """
    nodes = graph.nodes
    edges = list(graph.edges(data=True))  # Extract edges with attributes
    best_solution = {
        'total_cost': float('inf'),
        'relabilities': {node: 0 for node in nodes}
    }

    # Iterate over all possible subsets of edges. Begin with 1 edge.
    for num_edges in range(1, len(edges) + 1):
        # subset is a combination of edges with num_edges edges
        for subset in combinations(edges, num_edges):
            # Build a subgraph from the current subset
            subgraph = nx.Graph()
            # Populate the subgraph with the current subset of nodes
            subgraph.add_nodes_from(nodes)
            # Populate the subgraph with the current subset of edges
            subgraph.add_edges_from(
                [(u, v, data) for u, v, data in subset]
            )

            # Check connectivity of the subgraph
            if are_all_nodes_at_least_two_edges(subgraph):
                # Calculate cost and reliability
                total_cost = 0
                relabilities = {node: 0 for node in nodes}

                for u, v, data in subset:
                    edge_values = data['values']
                    total_cost += edge_values['cost']
                    relabilities[u] += edge_values['reliability']
                    relabilities[v] += edge_values['reliability']

                # Build solution
                current_solution = {
                    'total_cost': total_cost,
                    'relabilities': relabilities
                }

                # Check reliability
                if is_reliable(current_solution) and total_cost < best_solution['total_cost']:
                    best_solution = current_solution

    return best_solution


#### Testing for the brute force algorithm

**Unit testing**

In [13]:
# testing the function

# arrange
valid_solution = {
    'total_cost': 29,
    'relabilities': {
        0: 1.0,
        1: 1.0,
        2: 1.2,
        3: 1.0,
        4: 1.2
    }
}
graph = nx.Graph()
graph.add_nodes_from([0, 1, 2, 3, 4])
graph.add_edges_from([
    (0, 1, {"values": {"cost": 2, "reliability": 0.3}}),
    (0, 2, {"values": {"cost": 9, "reliability": 0.4}}),
    (0, 3, {"values": {"cost": 5, "reliability": 0.2}}),
    (0, 4, {"values": {"cost": 3, "reliability": 0.7}}),
    (1, 2, {"values": {"cost": 6, "reliability": 0.7}}),
    (1, 3, {"values": {"cost": 9, "reliability": 0.7}}),
    (1, 4, {"values": {"cost": 7, "reliability": 0.9}}),
    (2, 3, {"values": {"cost": 8, "reliability": 0.5}}),
    (2, 4, {"values": {"cost": 6, "reliability": 0.2}}),
    (3, 4, {"values": {"cost": 10, "reliability": 0.5}})
])

# act
solution = brute_force_reliable_network(graph)

# assert
valid_solution == solution

True

**Sample Solution with random graph generator**

In [14]:
# Generar grafo completamente conexo
num_nodes = 5
min_cost, max_cost = 1, 20

graph = generate_random_graph(num_nodes, min_cost, max_cost)

# Resolver el problema con fuerza bruta
solution = brute_force_reliable_network(graph)

print("Mejor solución encontrada:")
print("Costo total:", solution['total_cost'])
print("Fiabilidad por nodo:", solution['relabilities'])


Mejor solución encontrada:
Costo total: 45
Fiabilidad por nodo: {0: 1.5142998723589507, 1: 1.537987730617293, 2: 1.122014291298257, 3: 1.3804650502550428, 4: 1.606033428200495}
